In [ ]:
#  https://github.com/veeps/shelter_status

In [24]:
import datetime as dt
import pandas as pd
import re
import numpy as np
import nltk
import regex as reg
import matplotlib.pyplot as plt

from twitterscraper import query_tweets
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_multilabel_classification
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth',-1)
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 500)

## DATA CLEANING ALL LABELED TWEETS FIRE/FLOOD/HURRICANE

In [9]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/data'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data


In [10]:
df = pd.read_csv('all_tweets_clean.csv')

In [11]:
df1 = df[df.isna().any(axis=1)]

In [12]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()
print(df[df['disaster'].isnull()][null_columns])

       requesting_help disaster
10572 NaN               NaN    
13779 NaN               NaN    
18727 NaN               NaN    
28578 NaN               NaN    


In [13]:
df.iloc[10572]

text               one week after was hit by  of residents have no power have no clean water
requesting_help    NaN                                                                      
disaster           NaN                                                                      
languages          en                                                                       
Name: 10572, dtype: object

In [14]:
df.groupby('disaster')['requesting_help'].value_counts()

disaster   requesting_help
fire       0.0                468
           1.0                34 
floods     0.0                326
           1.0                153
hurricane  0.0                395
           1.0                109
Name: requesting_help, dtype: int64

In [15]:
df2 = df.dropna()

In [16]:
df3 =df2.reset_index()

In [17]:
df3=df2[['text','requesting_help','disaster']]

In [18]:
df3.reset_index(drop=True, inplace=True)

In [20]:
df3.loc[df3['requesting_help']==1]

,text,requesting_help,disaster
17,south florida about to have a very musical hurricane on our hands,1.0,hurricane
21,so will be im loaded with beer and water,1.0,hurricane
42,instead of having grey matter in brain he has straw full on brown matter from a donkey shat nuke hurricane dorian lunacy isn t a strong enough word if he only had a brain hatforbrains,1.0,hurricane
50,i was really hoping not to need my annual hurricane supplies list but looks to be heading this way stocking up today just in case,1.0,hurricane
64,a link to a gofundme for those being affected by the in the bahamas it has helped in the past with hurricane mathew please use your platform and help share it,1.0,hurricane
65,after disasters like it is inspiring to see individuals and organizations collaborating to meet the needs of those in crisis today we are thrilled to have partnered with and as we provided safe water for their work in the abacos,1.0,hurricane
68,family youve already contributed so much for us to aid those who were affected by if you would like to aid us simply lay your contribution at the altar tomorrow at am we ask that you would refrain from sending clothes however at fellowship bible baptist church,1.0,hurricane
71,we are doing our best to help the in their time of need please join us at don julio mexican grill,1.0,hurricane
78,thank you to west palm beach for donating over gallons of water for the bahamas,1.0,hurricane
80,as part of pbda s various relief efforts nd and rd grades made cards this week for the to deliver to families in the bahamas this weekend the cards will be included in care packages filled with food and hygiene products,1.0,hurricane


In [ ]:
index 17, 21, 42, 50, 107, 215, 220, 235, 237, 238, 
252, 253, 266, 274, 283, 299, 302, 303, 304,
328, 365, 368, 426, 459, 469, 471, 477, 489, 490, 514, 527, 528, 530, 532, 543, 581, 612, 633, 
636, 640, 641, 642, 645, 649, 651, 674, 696, 697, 727, 769, 784,
 yes 1 274, 636, 

In [22]:
pwd

'/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

In [25]:
df3.to_csv('df3_brian.csv')

In [26]:
pwd

'/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

In [28]:
df4 =pd.read_csv('df3_brian - df3_brian.csv')

In [31]:
df5 = df4[['text','requesting_help','disaster']]

In [32]:
df5

,text,requesting_help,disaster
0,a link to a gofundme for those being affected by the in the bahamas it has helped in the past with hurricane mathew please use your platform and help share it,1,hurricane
1,after disasters like it is inspiring to see individuals and organizations collaborating to meet the needs of those in crisis today we are thrilled to have partnered with and as we provided safe water for their work in the abacos,1,hurricane
2,family youve already contributed so much for us to aid those who were affected by if you would like to aid us simply lay your contribution at the altar tomorrow at am we ask that you would refrain from sending clothes however at fellowship bible baptist church,1,hurricane
3,we are doing our best to help the in their time of need please join us at don julio mexican grill,1,hurricane
4,thank you to west palm beach for donating over gallons of water for the bahamas,1,hurricane
5,as part of pbda s various relief efforts nd and rd grades made cards this week for the to deliver to families in the bahamas this weekend the cards will be included in care packages filled with food and hygiene products,1,hurricane
6,we re still collecting supplies to support our brothers sisters recently impacted by in the bahamas review the relief items needed and help us help them by picking up some of the items and dropping them off at the on tomorrow during worship,1,hurricane
7,forced blood drive cancellations donation center closures in impacted areas in times like these we call on donors across the u s to step up and help out to ensure blood needs can be met in storm impacted areas around the u s give,1,hurricane
8,thanks to all who came to top golf yesterday we were also able to gather a lot of donations for hurricane dorian relief,1,hurricane
9,how you can help in the wake of on the list donate blood learn more at,1,hurricane


In [33]:
X=df5['text']
y=df5['requesting_help']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=42)

In [35]:
pipe = Pipeline([
    ('tvec', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('svc', SVC(C=2,gamma='scale'))
])

params= {}

gs = GridSearchCV(pipe,params, cv=5, n_jobs=3)

In [36]:
X_train.shape

(1113,)

In [37]:
y_train.shape

(1113,)

In [38]:
gs.fit(X_train, y_train)
gs.score(X_train, y_train)

0.9991015274034142

In [39]:
gs.score(X_test, y_test)  # C=0.9, 80.1; C=2, 82.5; 

0.8387096774193549

In [41]:
pipe2 = Pipeline([
    ('tvec', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('lr', LogisticRegression(solver='liblinear'))
])

params= {}

gs2 = GridSearchCV(pipe2,params, cv=5, n_jobs=3)

In [42]:
gs2.fit(X_train, y_train)
gs2.score(X_train, y_train)

0.8373764600179695

In [43]:
gs2.score(X_test,y_test)

0.8413978494623656

In [44]:
gs2.best_params_

{}

In [ ]:
# https://stackoverflow.com/questions/44066264/how-to-choose-parameters-in-tfidfvectorizer-in-sklearn-during-unsupervised-clust
If you are, for instance, using these vectors in a classification task, you can vary these parameters (and of course also the parameters of the classifier) and see which values give you the best performance.

You can do that in sklearn easily with the GridSearchCV and Pipeline objects

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(MultinomialNB(
        fit_prior=True, class_prior=None))),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, train_y)

print("Best parameters set:")
print grid_search_tune.best_estimator_.steps

In [45]:
params5 = {
    'tfidf_max_df':(0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__estimator__alpha':(1e-2, 1e-3)
}

In [46]:
pipe3 = Pipeline([
    ('tvec', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('svc', SVC(C=2,gamma='scale'))
])



gs = GridSearchCV(pipe3,params5, cv=5, n_jobs=3)

In [47]:
gnb = GaussianNB()

In [ ]:
# Random forest